https://jump.dev/JuMP.jl/stable/tutorials/linear/introduction/

In [1]:
using IntervalArithmetic

In [2]:
include("cba.jl")

updatePCM_CBA_2 (generic function with 1 method)

In [3]:
include("cbaCombined.jl")

updatePCM_CBA_Combined (generic function with 1 method)

In [4]:
A = [
    (1..1, 1..1) (1.2..1.5, 1..2) (4..4.5, 3..5)
    (0.666666667..0.833333333, 0.5..1) (1..1, 1..1) (4.2..4.7, 4..5)
    (0.222222222..0.25, 0.2..0.333333333) (0.212765957..0.238095238, 0.2..0.25) (1..1, 1..1)
]
m, n = size(A)
showElements2(A)

[1.000, [1.000, 1.000], 1.000] [1.000, [1.200, 1.500], 2.000] [3.000, [4.000, 4.500], 5.000]
[0.500, [0.667, 0.833], 1.000] [1.000, [1.000, 1.000], 1.000] [4.000, [4.200, 4.700], 5.000]
[0.200, [0.222, 0.250], 0.333] [0.200, [0.213, 0.238], 0.250] [1.000, [1.000, 1.000], 1.000]


### LP 1
$$
\begin{align*}
    \text{minimize} ~~
    & \sum_{i \in N} \left( \varepsilon_i^\text{L} + \varepsilon_i^\text{U} \right), \\
    \text{subject to} ~~
    & a_{ij}^{\text{L}+}w_j^\text{U} - \varepsilon_i^\text{L} \leq
    w_i^\text{L} \leq
    a_{ij}^{\text{L}-}w_j^\text{U} + \varepsilon_i^\text{L}, ~~ i, j \in N, ~~ i \not= j, \\
    & a_{ij}^{\text{U}-}w_j^\text{L} - \varepsilon_i^\text{U} \leq
    w_i^\text{U} \leq
    a_{ij}^{\text{U}+}w_j^\text{L} + \varepsilon_i^\text{U}, ~~ i, j \in N, ~~ i \not= j, \\
    & \varepsilon_i^\text{L} \geq w_i^{\text{L}+} - w_i^{\text{L}-}, ~~
    \varepsilon_i^\text{U} \geq w_i^{\text{U}-} - w_i^{\text{U}+}, ~~ i \in N, \\
    & w_i^{\text{L}-} \leq a_{ij}^{\text{L}-}w_j^\text{U}, ~~
    w_i^{\text{L}+} \geq a_{ij}^{\text{L}+}w_j^\text{U}, ~~ i, j \in N, ~~ i \not= j, \\
    & w_i^{\text{U}-} \geq a_{ij}^{\text{U}-}w_j^\text{L}, ~~
    w_i^{\text{U}+} \leq a_{ij}^{\text{U}+}w_j^\text{L}, ~~ i, j \in N, ~~ i \not= j, \\
    & \sum_{i \in N\backslash\{ j \}} w_i^\text{U} + w_j^\text{L} \geq 1, ~~
    \sum_{i \in N\backslash\{ j \}} w_i^\text{L} + w_j^\text{U} \leq 1, ~~ j \in N, \\
    & w_i^\text{U} \geq w_i^\text{L} \geq \epsilon, ~~ i \in N, ~~
    w_i^{\text{L}+}, w_i^{\text{L}-}, \varepsilon_i^\text{L},
    w_i^{\text{U}-}, w_i^{\text{U}+}, \varepsilon_i^\text{U} \geq 0, ~~ i \in N
\end{align*}
$$

In [5]:
lpResult = @time solveLP_CBA(A)

Running HiGHS 1.3.0 [date: 1970-01-01, git hash: e5004072b-dirty]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
63 rows, 24 cols, 162 nonzeros
57 rows, 18 cols, 156 nonzeros
Presolve : Reductions: rows 57(-6); columns 18(-6); elements 156(-6)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     4.2885698801e-16 Pr: 3(3) 0s
         25     1.8008424238e-01 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 25
Objective value     :  1.8008424238e-01
HiGHS run time      :          0.00
  9.175854 seconds (17.79 M allocations: 1.037 GiB, 4.53% gc time, 99.78% compilation time)


([0.5329949238578681, 0.4467005076142132, 0.1116751269035533], [0.4467005076142132, 0.3553299492385787, 0.10659898477157362], [0.4467005076142132, 0.4467005076142132, 0.1065989847715736], [0.4467005076142132, 0.3553299494162437, 0.09504266099492385], [-0.0, 0.09137055819796952, 0.011556323776649752], [0.5329949238578681, 0.501015228426396, 0.1116751269035533], [0.5329949238578681, 0.4467005076142132, 0.08883248730964467], [-0.0, 0.054314720812182804, 0.02284263959390863], 0.1800842423807107)

### 手法1
LP 1を解いた後、次の方法でPCM $A$ を $\hat{A}$ に更新する．
$$
\hat{a}_{ij}^{\text{L}-} = \max\left( a_{ij}^{\text{L}-}, \frac{w_i^{\text{L}-} + \varepsilon_i^\text{L}}{w_j^{\text{U}-} - \varepsilon_j^\text{U}} \right), ~~
i, j \in N, ~~ i \not= j, \\
\hat{a}_{ij}^{\text{U}-} = \min\left( a_{ij}^{\text{U}-}, \frac{w_i^{\text{U}-} - \varepsilon_i^\text{U}}{w_j^{\text{L}-} + \varepsilon_j^\text{L}} \right), ~~
i, j \in N, ~~ i \not= j, \\
\hat{a}_{ij}^{\text{L}+} = \min\left( a_{ij}^{\text{L}+}, \frac{w_i^{\text{L}+} - \varepsilon_i^\text{L}}{w_j^{\text{U}+} + \varepsilon_j^\text{U}} \right), ~~
i, j \in N, ~~ i \not= j, \\
\hat{a}_{ij}^{\text{U}+} = \max\left( a_{ij}^{\text{U}+}, \frac{w_i^{\text{U}+} + \varepsilon_i^\text{U}}{w_j^{\text{L}+} - \varepsilon_j^\text{L}} \right), ~~
i, j \in N, ~~ i \not= j \\
$$

In [6]:
Â₁ = updatePCM_CBA_1(A, lpResult)
println("Â₁")
showElements2(Â₁)

Â₁
[1.000, [1.000, 1.000], 1.000] [0.892,       ∅       , 2.000] [3.000,       ∅       , 5.608]
[0.500,       ∅       , 1.122] [1.000, [1.000, 1.000], 1.000] [3.182,       ∅       , 5.271]
[0.178,       ∅       , 0.333] [0.190,       ∅       , 0.314] [1.000, [1.000, 1.000], 1.000]


In [7]:
println("W")
map(i -> lpResult[2][i]..lpResult[1][i], 1:n)

W


3-element Vector{Interval{Float64}}:
 [0.4467, 0.532995]
 [0.355329, 0.446701]
 [0.106598, 0.111676]

### 手法2
LP 1を解いた後、次の方法でPCM $A$ を $\hat{A}$ に更新する．
$$
\hat{a}_{ij}^{\text{L}-} = \max\left( a_{ij}^{\text{L}-}, \frac{w_i^{\text{L}+}}{w_j^\text{U}}, \frac{w_i^\text{L}}{w_j^{\text{U}+}} \right), ~~
i, j \in N, ~~ i \not= j, \\
\hat{a}_{ij}^{\text{U}-} = \min\left( a_{ij}^{\text{U}-}, \frac{w_i^{\text{U}+}}{w_j^\text{L}}, \frac{w_i^\text{U}}{w_j^{\text{L}+}} \right), ~~
i, j \in N, ~~ i \not= j, \\
\hat{a}_{ij}^{\text{L}+} = \min\left( a_{ij}^{\text{L}+}, \frac{w_i^{\text{L}-}}{w_j^\text{U}}, \frac{w_i^\text{L}}{w_j^{\text{U}-}} \right), ~~
i, j \in N, ~~ i \not= j, \\
\hat{a}_{ij}^{\text{U}+} = \max\left( a_{ij}^{\text{U}+}, \frac{w_i^{\text{U}-}}{w_j^\text{L}}, \frac{w_i^\text{U}}{w_j^{\text{L}-}} \right), ~~
i, j \in N, ~~ i \not= j \\
$$

In [8]:
Â₂ = updatePCM_CBA_2(A, lpResult)
println("Â₂")
showElements(Â₂)

Â₂
([1.000, 1.000], [1.000, 1.000]) (      ∅       , [0.892, 2.000]) (      ∅       , [3.000, 5.608])
(      ∅       , [0.500, 1.122]) ([1.000, 1.000], [1.000, 1.000]) (      ∅       , [3.182, 5.000])
(      ∅       , [0.178, 0.333]) (      ∅       , [0.200, 0.314]) ([1.000, 1.000], [1.000, 1.000])


### LP Combined
$$
\begin{align*}
    \text{minimize} ~~
    & \sum_{i, j \in N, ~ i \not= j} \left( \varepsilon_{ij}^{\text{L}-} + \varepsilon_{ij}^{\text{U}-} +
    \varepsilon_{ij}^{\text{L}+} + \varepsilon_{ij}^{\text{U}+} \right), \\
    \text{subject to} ~~
    & a_{ij}^{\text{L}+} w_j^\text{U} - \varepsilon_{ij}^{\text{L}+} \leq w_i^\text{L} \leq
    a_{ij}^{\text{L}-} w_j^\text{U} + \varepsilon_{ij}^{\text{L}-}, ~~
    i, j \in N, ~~ i \not= j, \\
    & a_{ij}^{\text{U}-} w_j^\text{L} - \varepsilon_{ij}^{\text{U}-} \leq w_i^\text{U} \leq
    a_{ij}^{\text{U}+} w_j^\text{L} + \varepsilon_{ij}^{\text{U}+}, ~~
    i, j \in N, ~~ i \not= j, \\
    & \sum_{j \in N\backslash\{ j \}} w_j^\text{U} + w_i^\text{L} \geq 1, ~~
    \sum_{j \in N\backslash\{ j \}} w_j^\text{L} + w_i^\text{U} \leq 1, ~~ i \in N, \\
    & w_i^\text{U} \geq w_i^\text{L} \geq \epsilon, ~~ i \in N, ~~
    \varepsilon_{ij}^{\text{L}-}, \varepsilon_{ij}^{\text{U}-}, \varepsilon_{ij}^{\text{L}+}, \varepsilon_{ij}^{\text{U}+} \geq 0, ~~
    i, j \in N, ~~ i \not= j
\end{align*}
$$

In [9]:
lpResult_Combined = @time solveLP_CBA_Combined(A)

Running HiGHS 1.3.0 [date: 1970-01-01, git hash: e5004072b-dirty]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
33 rows, 30 cols, 96 nonzeros
33 rows, 30 cols, 96 nonzeros
Presolve : Reductions: rows 33(-0); columns 30(-12); elements 96(-0)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     4.1040116848e-16 Pr: 3(3) 0s
         16     1.8008424238e-01 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 16
Objective value     :  1.8008424238e-01
HiGHS run time      :          0.00
  0.678414 seconds (1.27 M allocations: 65.387 MiB, 3.48% gc time, 99.77% compilation time)


([0.5329949238578681, 0.4467005076142132, 0.1116751269035533], [0.4467005076142132, 0.3553299492385787, 0.1065989847715736], [0.0 0.0 -0.0; -1.7766499382787515e-10 0.0 0.0; 0.0 0.011556323776649752 0.0], [0.0 0.0 0.0; 0.0 0.0 0.054314720812182804; 0.0 0.0 0.0], [0.0 -0.0 0.0; 0.0 0.0 0.09137055837563451; 0.0 0.0 0.0], [0.0 0.0 -0.0; 0.0 0.0 0.0; 0.0 0.02284263959390863 0.0], 0.1800842423807107)

In [10]:
println("W")
map(i -> lpResult_Combined[2][i]..lpResult_Combined[1][i], 1:n)

W


3-element Vector{Interval{Float64}}:
 [0.4467, 0.532995]
 [0.355329, 0.446701]
 [0.106598, 0.111676]

LP Combinedを解いた後、次の方法でPCM $A$ を $\hat{A}$ に更新する．
$$
\hat{a}_{ij}^{\text{L}-} = \max\left( a_{ij}^{\text{L}-} + \frac{\varepsilon_{ij}^{\text{L}-}}{w_j^\text{U}}, ~~
\left( a_{ji}^{\text{U}-} - \frac{\varepsilon_{ji}^{\text{U}-}}{w_i^\text{L}} \right)^{-1} \right), ~~
i, j \in N, ~~ i \not= j, \\
\hat{a}_{ij}^{\text{U}-} = \min\left( a_{ij}^{\text{U}-} - \frac{\varepsilon_{ij}^{\text{U}-}}{w_j^\text{L}}, ~~
\left( a_{ji}^{\text{L}-} + \frac{\varepsilon_{ji}^{\text{L}-}}{w_i^\text{U}} \right)^{-1} \right), ~~
i, j \in N, ~~ i \not= j, \\
\hat{a}_{ij}^{\text{L}+} = \min\left( a_{ij}^{\text{L}+} - \frac{\varepsilon_{ij}^{\text{L}+}}{w_j^\text{U}}, ~~
\left( a_{ji}^{\text{U}+} + \frac{\varepsilon_{ji}^{\text{U}+}}{w_i^\text{L}} \right)^{-1} \right), ~~
i, j \in N, ~~ i \not= j, \\
\hat{a}_{ij}^{\text{U}+} = \max\left( a_{ij}^{\text{U}+} + \frac{\varepsilon_{ij}^{\text{U}+}}{w_j^\text{U}}, ~~
\left( a_{ji}^{\text{L}+} - \frac{\varepsilon_{ji}^{\text{L}+}}{w_i^\text{U}} \right)^{-1} \right), ~~
i, j \in N, ~~ i \not= j, \\
$$

In [11]:
Â₃ = updatePCM_CBA_Combined(A, lpResult_Combined)
println("Â₃")
showElements(Â₃)

Â₃
([1.000, 1.000], [1.000, 1.000]) ([1.200, 1.500], [1.000, 2.000]) ([4.000, 4.500], [3.000, 5.000])
([0.667, 0.833], [0.500, 1.000]) ([1.000, 1.000], [1.000, 1.000]) (      ∅       , [3.182, 5.000])
([0.222, 0.250], [0.200, 0.333]) (      ∅       , [0.200, 0.314]) ([1.000, 1.000], [1.000, 1.000])


In [12]:
println("Â₁")
showElements(Â₁)
println("\nÂ₂")
showElements(Â₂)
println("\nÂ₃")
showElements(Â₃)

Â₁
([1.000, 1.000], [1.000, 1.000]) (      ∅       , [0.892, 2.000]) (      ∅       , [3.000, 5.608])
(      ∅       , [0.500, 1.122]) ([1.000, 1.000], [1.000, 1.000]) (      ∅       , [3.182, 5.271])
(      ∅       , [0.178, 0.333]) (      ∅       , [0.190, 0.314]) ([1.000, 1.000], [1.000, 1.000])

Â₂
([1.000, 1.000], [1.000, 1.000]) (      ∅       , [0.892, 2.000]) (      ∅       , [3.000, 5.608])
(      ∅       , [0.500, 1.122]) ([1.000, 1.000], [1.000, 1.000]) (      ∅       , [3.182, 5.000])
(      ∅       , [0.178, 0.333]) (      ∅       , [0.200, 0.314]) ([1.000, 1.000], [1.000, 1.000])

Â₃
([1.000, 1.000], [1.000, 1.000]) ([1.200, 1.500], [1.000, 2.000]) ([4.000, 4.500], [3.000, 5.000])
([0.667, 0.833], [0.500, 1.000]) ([1.000, 1.000], [1.000, 1.000]) (      ∅       , [3.182, 5.000])
([0.222, 0.250], [0.200, 0.333]) (      ∅       , [0.200, 0.314]) ([1.000, 1.000], [1.000, 1.000])


In [13]:
println("Â₁")
showElements2(Â₁)
println("\nÂ₂")
showElements2(Â₂)
println("\nÂ₃")
showElements2(Â₃)

Â₁
[1.000, [1.000, 1.000], 1.000] [0.892,       ∅       , 2.000] [3.000,       ∅       , 5.608]
[0.500,       ∅       , 1.122] [1.000, [1.000, 1.000], 1.000] [3.182,       ∅       , 5.271]
[0.178,       ∅       , 0.333] [0.190,       ∅       , 0.314] [1.000, [1.000, 1.000], 1.000]

Â₂
[1.000, [1.000, 1.000], 1.000] [0.892,       ∅       , 2.000] [3.000,       ∅       , 5.608]
[0.500,       ∅       , 1.122] [1.000, [1.000, 1.000], 1.000] [3.182,       ∅       , 5.000]
[0.178,       ∅       , 0.333] [0.200,       ∅       , 0.314] [1.000, [1.000, 1.000], 1.000]

Â₃
[1.000, [1.000, 1.000], 1.000] [1.000, [1.200, 1.500], 2.000] [3.000, [4.000, 4.500], 5.000]
[0.500, [0.667, 0.833], 1.000] [1.000, [1.000, 1.000], 1.000] [3.182,       ∅       , 5.000]
[0.200, [0.222, 0.250], 0.333] [0.200,       ∅       , 0.314] [1.000, [1.000, 1.000], 1.000]


In [14]:
println("Â₁")
showWidths(Â₁)
println("\nÂ₂")
showWidths(Â₂)
println("\nÂ₃")
showWidths(Â₃)

Â₁
(0.00000, 0.00000) ( undef , 1.10841) ( undef , 2.60795)
( undef , 0.62159) (0.00000, 0.00000) ( undef , 2.08966)
( undef , 0.15502) ( undef , 0.12459) (0.00000, 0.00000)

Â₂
(0.00000, 0.00000) ( undef , 1.10841) ( undef , 2.60795)
( undef , 0.62159) (0.00000, 0.00000) ( undef , 1.81818)
( undef , 0.15502) ( undef , 0.11429) (0.00000, 0.00000)

Â₃
(0.00000, 0.00000) (0.30000, 1.00000) (0.50000, 2.00000)
(0.16667, 0.50000) (0.00000, 0.00000) ( undef , 1.81818)
(0.02778, 0.13333) ( undef , 0.11429) (0.00000, 0.00000)
